# EfficientNetB0

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

image_folder = '/content/drive/MyDrive/2024-2025/ELEC 594/Audio Dataset/training_img_spectrograms'
label_file = '/content/drive/MyDrive/2024-2025/ELEC 594/Audio Dataset/labels.csv'

labels_df = pd.read_csv(label_file)
labels_df['file_name'] = labels_df['file_name'] + '.png'
labels_df['target'] = labels_df['target'].astype(str)

# data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Training and Validation Generators
train_gen = datagen.flow_from_dataframe(
    labels_df,
    directory=image_folder,
    x_col='file_name',
    y_col='target',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_dataframe(
    labels_df,
    directory=image_folder,
    x_col='file_name',
    y_col='target',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load Pre-trained EfficientNet Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # Freeze the base model

# Build the Model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)


In [ ]:
# Evaluate the Model and Get Predictions
val_gen.reset()
predictions = model.predict(val_gen)
predicted_labels = (predictions > 0.5).astype(int)
true_labels = val_gen.classes

fpr, tpr, _ = roc_curve(true_labels, predictions)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

cm = confusion_matrix(true_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Noisy', 'Noisy'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

# RCN

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

image_folder = '/content/drive/MyDrive/2024-2025/ELEC 594/Audio Dataset/training_img_spectrograms'
label_file = '/content/drive/MyDrive/2024-2025/ELEC 594/Audio Dataset/labels.csv'

labels_df = pd.read_csv(label_file)
labels_df['file_name'] = labels_df['file_name'] + '.png'
labels_df['target'] = labels_df['target'].astype(str)

# data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=5,
    width_shift_range=0.02,
    height_shift_range=0.02,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

#Training and Validation Generators
train_gen = datagen.flow_from_dataframe(
    labels_df,
    directory=image_folder,
    x_col='file_name',
    y_col='target',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_dataframe(
    labels_df,
    directory=image_folder,
    x_col='file_name',
    y_col='target',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

# Load Pre-trained ResNet Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Build the Model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),  # Dropout to prevent overfitting
    layers.Dense(1, activation='sigmoid')
])

# Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

# Unfreeze layers of the base model
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompile the Model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Continue Training the Model
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)


In [ ]:
# Evaluate the Model
val_gen.reset()
predictions = model.predict(val_gen)
predicted_labels = (predictions > 0.5).astype(int)
true_labels = val_gen.classes

fpr, tpr, _ = roc_curve(true_labels, predictions)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

cm = confusion_matrix(true_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Noisy', 'Noisy'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()